**References:**  

https://www.datacamp.com/community/tutorials/finance-python-trading

https://github.com/datacamp/datacamp-community-tutorials/blob/master/Python%20Finance%20Tutorial%20For%20Beginners/Python%20For%20Finance%20Beginners%20Tutorial.ipynb

https://pypi.python.org/pypi/fix-yahoo-finance

http://www.learndatasci.com/python-finance-part-yahoo-finance-api-pandas-matplotlib/

In [4]:
# Import initial libraries

import pandas as pd
# import pandas_gbq
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline


ModuleNotFoundError: No module named 'pandas_gbq'

In [11]:
sql = """
    SELECT name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE state = 'TX'
    LIMIT 100
"""
df = pd.read_gbq(sql, dialect='standard')

ValueError: Could not determine project ID and one was not supplied.

In [2]:
# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

4.6.0


In [3]:
# Import the Sample worksheet with acquisition dates and initial cost basis:

portfolio_df = pd.read_excel('Sample stocks acquisition dates_costs.xlsx')
all_columns = ['Acquisition Date', 'Ticker','Quantity','Unit Cost', 'Cost Basis','Currency','Transaction Cost']
all_rows = [
    ['2018-09-28','GOOGL',1,1202,1202,'USD',19.21],
    ['2018-09-28','ZEN',3,70.25,210.75,'USD',15.74]
]
portfolio_df = pd.DataFrame(all_rows, columns= all_columns)
portfolio_df['Acquisition Date'].astype('datetime64[ns]')
portfolio_df.head(10)

,Acquisition Date,Ticker,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost
0,2018-09-28,GOOGL,1,1202.00,1202.00,USD,19.21
1,2018-09-28,ZEN,3,70.25,210.75,USD,15.74


In [4]:
# Confirm that you have 8 values for each column.

portfolio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Acquisition Date  2 non-null      object 
 1   Ticker            2 non-null      object 
 2   Quantity          2 non-null      int64  
 3   Unit Cost         2 non-null      float64
 4   Cost Basis        2 non-null      float64
 5   Currency          2 non-null      object 
 6   Transaction Cost  2 non-null      float64
dtypes: float64(3), int64(1), object(3)
memory usage: 240.0+ bytes


In [5]:
from pandas.tseries.offsets import BDay
# Date Ranges for SP 500 and for all tickers
start_sp = datetime.datetime(2018, 9, 28)
end_sp = datetime.datetime.today().date()
last_BD = (end_sp - BDay(1)).date()
print(last_BD)

2020-05-01


In [6]:
# Leveraged from the helpful Datacamp Python Finance trading blog post.

from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)
spp500 = yf.Ticker('^GSPC')
sp500 = pdr.get_data_yahoo('^GSPC', 
                           start_sp,
                             end_sp)

                
sp500.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-09-27,2911.649902,2927.219971,2909.270020,2914.000000,2914.000000,3060850000
2018-09-28,2910.030029,2920.530029,2907.500000,2913.979980,2913.979980,3432300000
2018-10-01,2926.290039,2937.060059,2917.909912,2924.590088,2924.590088,3364190000
2018-10-02,2923.800049,2931.419922,2919.370117,2923.429932,2923.429932,3401880000
2018-10-03,2931.689941,2939.860107,2921.360107,2925.510010,2925.510010,3598710000


In [7]:
# Create a dataframe with only the Adj Close column as that's all we need for this analysis.

sp_500_adj_close = sp500[['Adj Close']].reset_index()

In [8]:
# Stock comparison code
def get(tickers):
    def data(ticker):
        return yf.Ticker(ticker).history(period = "max")
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))

tickers = portfolio_df['Ticker'].unique()          
all_data = get(tickers)
all_data.head()

Open   High    Low  Close    Volume  Dividends  \
Ticker Date                                                          
GOOGL  2004-08-19  50.05  52.08  48.03  50.22  44659000          0   
       2004-08-20  50.56  54.59  50.30  54.21  22834300          0   
       2004-08-23  55.43  56.80  54.58  54.75  18256100          0   
       2004-08-24  55.68  55.86  51.84  52.49  15247300          0   
       2004-08-25  52.53  54.05  51.99  53.05   9188600          0   

                   Stock Splits  
Ticker Date                      
GOOGL  2004-08-19           0.0  
       2004-08-20           0.0  
       2004-08-23           0.0  
       2004-08-24           0.0  
       2004-08-25           0.0

In [9]:
# Also only pulling the ticker, date and adj. close columns for our tickers.
adj_close = all_data[['Close']].reset_index()
adj_close = adj_close.rename(columns = {'Close' : 'Adj Close'})
adj_close_latest = adj_close[adj_close['Date']==str(last_BD)]
adj_close_latest.set_index('Ticker', inplace=True)
portfolio_df.set_index(['Ticker'], inplace=True)

In [10]:
# Merge the portfolio dataframe with the adj close dataframe; they are being joined by their indexes.portfolio_df.set_index(['Ticker'], inplace=True)
merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)
merged_portfolio.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost,Date,Adj Close
Ticker,,,,,,,,
GOOGL,2018-09-28,1,1202.00,1202.00,USD,19.21,2020-05-01,1317.32
ZEN,2018-09-28,3,70.25,210.75,USD,15.74,2020-05-01,72.55


In [11]:
# The below creates a new column which is the ticker return; takes the latest adjusted close for each position
# and divides that by the initial share cost.
merged_portfolio['ticker return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] - 1
merged_portfolio['Acquisition Date'] = merged_portfolio['Acquisition Date'].astype('datetime64[ns]')

merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost,Date,Adj Close,ticker return
Ticker,,,,,,,,,
GOOGL,2018-09-28,1,1202.00,1202.00,USD,19.21,2020-05-01,1317.32,0.09594
ZEN,2018-09-28,3,70.25,210.75,USD,15.74,2020-05-01,72.55,0.03274


In [13]:
# Here we are merging the new dataframe with the sp500 adjusted closes since the sp start price based on 
# each ticker's acquisition date and sp500 close date.
merged_portfolio.dtypes
merged_portfolio_sp = pd.merge(merged_portfolio, sp_500_adj_close, left_on='Acquisition Date', right_on='Date')
del merged_portfolio_sp['Date_y']
merged_portfolio_sp.rename(columns={'Date_x': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close'
                                    , 'Adj Close_y': 'SP 500 Initial Close'}, inplace=True)
merged_portfolio_sp.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close
0,2018-09-28,1,1202.00,1202.00,USD,19.21,2020-05-01,1317.32,0.09594,2913.97998
1,2018-09-28,3,70.25,210.75,USD,15.74,2020-05-01,72.55,0.03274,2913.97998


In [14]:
# This new column determines what SP 500 equivalent purchase would have been at purchase date of stock.
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['SP 500 Initial Close']
merged_portfolio_sp.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares
0,2018-09-28,1,1202.00,1202.00,USD,19.21,2020-05-01,1317.32,0.09594,2913.97998,0.412494
1,2018-09-28,3,70.25,210.75,USD,15.74,2020-05-01,72.55,0.03274,2913.97998,0.072324


In [15]:
# We are joining the developing dataframe with the sp500 closes again, this time with the latest close for SP.
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp_500_adj_close, left_on='Latest Date', right_on='Date')
del merged_portfolio_sp_latest['Date']
merged_portfolio_sp_latest.rename(columns={'Adj Close': 'SP 500 Latest Close'}, inplace=True)

merged_portfolio_sp_latest.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close
0,2018-09-28,1,1202.00,1202.00,USD,19.21,2020-05-01,1317.32,0.09594,2913.97998,0.412494,2830.709961
1,2018-09-28,3,70.25,210.75,USD,15.74,2020-05-01,72.55,0.03274,2913.97998,0.072324,2830.709961


In [16]:
# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest['SP Return'] = merged_portfolio_sp_latest['SP 500 Latest Close'] / merged_portfolio_sp_latest['SP 500 Initial Close'] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['ticker return'] - merged_portfolio_sp_latest['SP Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['SP 500 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['SP 500 Latest Close']

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['SP 500 Value']

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest['SP 500 Gain / (Loss)'] = merged_portfolio_sp_latest['SP 500 Value'] - merged_portfolio_sp_latest['Cost Basis']

merged_portfolio_sp_latest.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss)
0,2018-09-28,1,1202.00,1202.00,USD,19.21,2020-05-01,1317.32,0.09594,2913.97998,0.412494,2830.709961,-0.028576,0.124516,1317.32,1167.651595,149.668405,115.32,-34.348405
1,2018-09-28,3,70.25,210.75,USD,15.74,2020-05-01,72.55,0.03274,2913.97998,0.072324,2830.709961,-0.028576,0.061316,217.65,204.727599,12.922401,6.90,-6.022401


In [17]:
# Cumulative sum of original investment
merged_portfolio_sp_latest['Cum Invst'] = merged_portfolio_sp_latest['Cost Basis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest['Cum Ticker Returns'] = merged_portfolio_sp_latest['Ticker Share Value'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest['Cum SP Returns'] = merged_portfolio_sp_latest['SP 500 Value'].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest['Cum Ticker Returns'] / merged_portfolio_sp_latest['Cum Invst']

merged_portfolio_sp_latest.head()


,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult
0,2018-09-28,1,1202.00,1202.00,USD,19.21,2020-05-01,1317.32,0.09594,2913.97998,...,0.124516,1317.32,1167.651595,149.668405,115.32,-34.348405,1202.00,1317.32,1167.651595,1.095940
1,2018-09-28,3,70.25,210.75,USD,15.74,2020-05-01,72.55,0.03274,2913.97998,...,0.061316,217.65,204.727599,12.922401,6.90,-6.022401,1412.75,1534.97,1372.379194,1.086512


In [33]:
merged_portfolio_sp_latest.to_csv('Portfolio_returns.csv')

## Assessing Where Positions are At versus Highest Close

In [18]:
# Referencing the adj_close dataframe from above

adj_close.head()

,Ticker,Date,Adj Close
0,GOOGL,2004-08-19,50.22
1,GOOGL,2004-08-20,54.21
2,GOOGL,2004-08-23,54.75
3,GOOGL,2004-08-24,52.49
4,GOOGL,2004-08-25,53.05


In [19]:
portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost
Ticker,,,,,,
GOOGL,2018-09-28,1,1202.00,1202.00,USD,19.21
ZEN,2018-09-28,3,70.25,210.75,USD,15.74


In [20]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

portfolio_df.reset_index(inplace=True)

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on='Ticker')

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Quantity,Unit Cost,Cost Basis,Currency,Transaction Cost
0,GOOGL,2004-08-19,50.22,2018-09-28,1,1202.0,1202.0,USD,19.21
1,GOOGL,2004-08-20,54.21,2018-09-28,1,1202.0,1202.0,USD,19.21
2,GOOGL,2004-08-23,54.75,2018-09-28,1,1202.0,1202.0,USD,19.21
3,GOOGL,2004-08-24,52.49,2018-09-28,1,1202.0,1202.0,USD,19.21
4,GOOGL,2004-08-25,53.05,2018-09-28,1,1202.0,1202.0,USD,19.21


In [21]:
# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']

del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']


# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)
adj_close_acq_date

,Ticker,Date,Adj Close,Acquisition Date,Currency,Transaction Cost
0,GOOGL,2004-08-19,50.22,2018-09-28,USD,19.21
1,GOOGL,2004-08-20,54.21,2018-09-28,USD,19.21
2,GOOGL,2004-08-23,54.75,2018-09-28,USD,19.21
3,GOOGL,2004-08-24,52.49,2018-09-28,USD,19.21
4,GOOGL,2004-08-25,53.05,2018-09-28,USD,19.21
...,...,...,...,...,...,...
5450,ZEN,2020-04-27,73.74,2018-09-28,USD,15.74
5451,ZEN,2020-04-28,71.91,2018-09-28,USD,15.74
5452,ZEN,2020-04-29,75.87,2018-09-28,USD,15.74
5453,ZEN,2020-04-30,76.88,2018-09-28,USD,15.74


In [22]:
# Anything less than 0 means that the stock close was prior to acquisition.
adj_close_acq_date['Date Delta'] = adj_close_acq_date['Date'] - adj_close_acq_date['Acquisition Date'].astype('datetime64[ns]')

adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)  

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Currency,Transaction Cost,Date Delta
0,GOOGL,2004-08-19,50.22,2018-09-28,USD,19.21,-445219200000000000
1,GOOGL,2004-08-20,54.21,2018-09-28,USD,19.21,-445132800000000000
2,GOOGL,2004-08-23,54.75,2018-09-28,USD,19.21,-444873600000000000
3,GOOGL,2004-08-24,52.49,2018-09-28,USD,19.21,-444787200000000000
4,GOOGL,2004-08-25,53.05,2018-09-28,USD,19.21,-444700800000000000


In [23]:
# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).

adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>=0]

adj_close_acq_date_modified.head()

,Ticker,Date,Adj Close,Acquisition Date,Currency,Transaction Cost,Date Delta
3553,GOOGL,2018-09-28,1207.08,2018-09-28,USD,19.21,0
3554,GOOGL,2018-10-01,1208.53,2018-09-28,USD,19.21,259200000000000
3555,GOOGL,2018-10-02,1207.64,2018-09-28,USD,19.21,345600000000000
3556,GOOGL,2018-10-03,1211.53,2018-09-28,USD,19.21,432000000000000
3557,GOOGL,2018-10-04,1177.07,2018-09-28,USD,19.21,518400000000000


In [24]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values='Adj Close', aggfunc=np.max)

adj_close_pivot.reset_index(inplace=True)

adj_close_pivot

,Ticker,Acquisition Date,Adj Close
0,GOOGL,2018-09-28,1524.87
1,ZEN,2018-09-28,93.74


In [25]:
# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close
                                             , on=['Ticker', 'Adj Close'])

adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,GOOGL,2018-09-28,1524.87,2020-02-19
1,ZEN,2018-09-28,93.74,2019-07-26


In [26]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.

merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged
                                             , on=['Ticker', 'Acquisition Date'])

# Renaming so that it's clear that the new columns are two year closing high and two year closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

merged_portfolio_sp_latest_YTD_sp_closing_high

NameError: name 'merged_portfolio_sp_latest_YTD_sp' is not defined

In [27]:
# Not needed for this blog post -- this is if you have multiple positions for the same ticker with different acquisition dates.
# merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'] = merged_portfolio_sp_latest_YTD_sp_closing_high.index

# merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker #'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'].map(str) + ' ' + merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'].map(str)

# merged_portfolio_sp_latest_YTD_sp_closing_high.head()

## YTD and Trailing Stop Charts

In [28]:
# Ploty is an outstanding resource for interactive charts.

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['Share YTD'][0:10],
    name = 'Ticker YTD')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'][0:10],
    name = 'SP500 YTD')
    
data = [trace1, trace2]

layout = go.Layout(title = 'YTD Return vs S&P 500 YTD'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

NameError: name 'merged_portfolio_sp_latest_YTD_sp' is not defined

In [29]:
# Current Share Price versus Closing High Since Purchased
import nbformat
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')
    
data = [trace1]

layout = go.Layout(title = 'Adj Close % off of High'
    , barmode = 'group'
    , yaxis=dict(title='% Below Adj Close High', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

NameError: name 'merged_portfolio_sp_latest_YTD_sp_closing_high' is not defined

## Total Return Comparison Charts

In [30]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP Return'][0:10],
    name = 'SP500 Total Return')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker', tickformat=".2%")
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

NameError: name 'merged_portfolio_sp_latest_YTD_sp_closing_high' is not defined

## Cumulative Returns Over Time

In [31]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return ($)')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP 500 Gain / (Loss)'][0:10],
    name = 'SP 500 Total Return ($)')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return %',
    yaxis='y2')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'Gain / (Loss) Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Gain / (Loss) ($)')
    , yaxis2=dict(title='Ticker Return', overlaying='y', side='right', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.75,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

NameError: name 'merged_portfolio_sp_latest_YTD_sp_closing_high' is not defined

In [32]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Invst'],
    # mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum SP Returns'],
    # mode = 'lines+markers',
    name = 'Cum SP500 Returns')

trace3 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker Returns'],
    # mode = 'lines+markers',
    name = 'Cum Ticker Returns')

trace4 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker ROI Mult'],
    # mode = 'lines+markers',
    name = 'Cum ROI Mult'
    , yaxis='y2')


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(title = 'Total Cumulative Investments Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.4,y=1)
    , yaxis2=dict(title='Cum ROI Mult', overlaying='y', side='right')               
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

NameError: name 'merged_portfolio_sp_latest_YTD_sp_closing_high' is not defined